In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
cd drive

[Errno 2] No such file or directory: 'drive'
/content/drive


In [0]:
import torch 
import numpy as np 
import torchvision.transforms as transforms 
import torchvision 
import torch.nn.functional as FUNC
import pandas as pd
import os
from PIL import Image

In [0]:
class simple_block(torch.nn.Module):
    #No expansion in base block so value is 1
    expansion = 1
    #planes: desired output plane
    #dim_change: Change the value if the input shape has to change
    def __init__(self,input_planes,planes,stride=1,dim_change=None):
        super(simple_block,self).__init__()
        
        #declare convolutional layers with batch norms
        #In ResNet18, each residual block has 1 convolution layer and 1 batch normalization layer, twice
        #Values of kernal size and stride according to paper
        self.conv_1 = torch.nn.Conv2d(input_planes,planes,stride=stride,kernel_size=3,padding=1)
        #Batch Normalization
        self.bn_1   = torch.nn.BatchNorm2d(planes)
        self.conv_2 = torch.nn.Conv2d(planes,planes,stride=1,kernel_size=3,padding=1)
        self.bn_2   = torch.nn.BatchNorm2d(planes)
        #If the input size has to be changed then change this value
        self.dim_change = dim_change
    def forward(self,x):
        #residue
        res = x
        #First output: first convolve then batch normalize
        output = FUNC.relu(self.bn_1(self.conv_1(x)))
        #Next connected layer
        output = self.bn_2(self.conv_2(output))
        #Change the dimensions if needed
        if self.dim_change is not None:
            res = self.dim_change(res)
        #Add the residue with this output
        output += res
        #Then pass through ReLU
        output = FUNC.relu(output)

        return output


In [0]:
class ResNet(torch.nn.Module):
  #num_layers: number of times of each block
    def __init__(self,block,num_layers,classes=8):
        super(ResNet,self).__init__()
        #First convolution has 64 layers
        #according to research paper:
        self.input_planes = 64
        #Converts 3 channels to 64 channels
        self.conv_1 = torch.nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1)
        self.bn_1   = torch.nn.BatchNorm2d(64)
        #num_layer = [2,2,2,2] for ResNet18
        self.layer_1 = self._layer(block,64,num_layers[0],stride=1)
        self.layer_2 = self._layer(block,128,num_layers[1],stride=2)
        self.layer_3 = self._layer(block,256,num_layers[2],stride=2)
        self.layer_4 = self._layer(block,512,num_layers[3],stride=2)
        #Average pool using 4x4 filter size with stride 1
        self.averagePool = torch.nn.AdaptiveAvgPool2d((1,1))
        #Fully connected layer has 512 times the number of times the block was extended
        self.f_c    =  torch.nn.Linear(512*block.expansion,classes)
    
    #planes: Number of layers you want to have in the block
    def _layer(self,block,planes,num_layers,stride=1):
        dim_change = None
        if stride!=1 or planes != self.input_planes*block.expansion:
            dim_change = torch.nn.Sequential(torch.nn.Conv2d(self.input_planes,planes*block.expansion,kernel_size=1,stride=stride),
                                             torch.nn.BatchNorm2d(planes*block.expansion))
        #Using sequential the dimensions can be changed
        Layers =[]
        #Append the first block
        Layers.append(block(self.input_planes,planes,stride=stride,dim_change=dim_change))
        #The input size changes due to above layer
        self.input_planes = planes * block.expansion
        for i in range(1,num_layers):
            Layers.append(block(self.input_planes,planes))
            self.input_planes = planes * block.expansion
        
        #Converting all the Layers to fully connected layer
        return torch.nn.Sequential(*Layers)

    def forward(self,x):
        x = FUNC.relu(self.bn_1(self.conv_1(x)))

        x = self.layer_1(x)
        x = self.layer_2(x)
        x = self.layer_3(x)
        x = self.layer_4(x)

        x = self.averagePool(x)
        x = x.view(x.size(0),-1)
        x = self.f_c(x)
        return x


In [0]:
class FaceLandmarksDataset():
    """Face Landmarks dataset."""

    def __init__(self,root_dir,total_count, csv_file, transform=None):
        
        self.root_dir = root_dir
        self.total_count = total_count
        self.csv_file = np.array(pd.read_csv(csv_file,header=None))[0]
        self.transform = transform

    def __len__(self):
        return self.total_count

    def __getitem__(self, image_no):
        img_name = os.path.join(self.root_dir,
                                str(image_no+1)+".jpg")
        image = Image.open(img_name)
        sample = {'image': image, 'label': self.csv_file[image_no]-1}

        if self.transform:
            sample['image'] = self.transform(sample['image'])

        return sample    

In [0]:
def test():
        #To convert data from PIL to tensor
    transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
        )

    
    trainDataset = FaceLandmarksDataset('/content/drive/My Drive/HW3_data/train',1888,'/content/drive/My Drive/HW3_data/train_labels.csv',transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]))

    testDataset = FaceLandmarksDataset('/content/drive/My Drive/HW3_data/test',800,'/content/drive/My Drive/HW3_data/test_labels.csv',transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]))
    trainDataloader = torch.utils.data.DataLoader(trainDataset,batch_size = 8, shuffle = True, num_workers=4)
    testDataloader = torch.utils.data.DataLoader(testDataset,batch_size = 8, shuffle = True, num_workers=4)
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)

    #ResNet-18 
    net = ResNet(simple_block,[2,2,2,2],8)

    net.to(device)
    costFunc = torch.nn.CrossEntropyLoss()
    optimizer =  torch.optim.SGD(net.parameters(),lr=0.02,momentum=0.9)

    for epoch in range(5):
        print("epoch no:"+str(epoch))
        closs = 0
        for i,batch in enumerate(trainDataloader,0):
            data,output = batch['image'],batch['label']
            data,output = data.to(device),output.to(device)
            predict = net(data)
            loss = costFunc(predict,output)
            closs = loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #print every 1000th time
            if i%100 == 0:
                print('[epoch no:%d  batch no:%d] loss: %.5f'% (epoch+1,i+1,closs/1000))
                closs = 0
        correct_Hits=0
        total=0
        for batches in testDataloader:
            data,output = batches['image'],batches['label']
            data,output = data.to(device),output.to(device)
            predict = net(data)
            _,predict = torch.max(predict.data,1)  #returns max as well as its index
            total += output.size(0)
            correct_Hits += (predict==output).sum().item()
        print('Accuracy on epoch ',epoch+1,'= ',str((correct_Hits/total)*100))
        print("------------------------------------------------------")

    correct_Hits=0
    total=0
    for batches in testDataloader:
        data,output = batches['image'],batches['label']
        data,output = data.to(device),output.to(device)
        predict = net(data)
        _,predict = torch.max(predict.data,1)  #returns max as well as its index
        total += output.size(0)
        correct_Hits += (predict==output).sum().item()
    print('Accuracy = '+str((correct_Hits/total)*100))


test()